# Real program

Structure of url :
https://www.leboncoin.fr/annonces/offres/rhone_alpes/?th=1&q=batterie%20lithium&it=0&parrot=1
region : /rhone_alpes/
keywords : q=batterie%20lithium
look only in title : it=1
only urgent ones : ur=1
category : https://www.leboncoin.fr/locations/offres/rhone_alpes/?th=1&q=batterie%20lithium&parrot=1
city : location=Toutes%20les%20communes%2073000


In [1]:
from urllib.request import urlopen
from lxml import etree

# ====== we define the parameters of the request

search = '%20'.join(['batterie', 'lithium'])
search = '%20'.join(['batterie', 'ordinateur'])

regions = ["rhone_alpes", "provence_alpes_cote_d_azur", "auvergne", "ile_de_france", "midi_pyrenees", 'languedoc_roussillon']
region = regions[4]
code_postal = "73000"
#location = "&location=Toutes%20les%20communes%20" + code_postal
location = ""


In [218]:
import re # regular expressions

class ad_class:
    def __init__(self, title, price, url, time, city, img):
        self.title = title
        self.price = price
        self.url = url
        self.time = time
        self.city = city
        self.img = img
        
        # we try to get the capacity from the title
        ah = self.return_ah(self.title)
        # if we don't succeed, we try from the text of the ad
        if (ah == None):
            page_details = detailed_page_class(self.url)
            ah = self.return_ah(self.desc)
        # if we don't succeed, we try to find the info in the pictures
        if (ah == None):
            pass
        self.ah = ah
    # first evaluation to see if the ad is interesting
    # returns True if the ad is interesting
    def looks_interesting(self):
        if float(self.price) > 300:
            return False
        print(str(self.ah / self.price) + " ah/euros")
        return True
    def return_ah(self, string):
        pattern = re.compile(r'([0-9.,]*\Smah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-3].replace(",", ".")
            return float(ah)/1000
        pattern = re.compile(r'([0-9.,]* mah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-4].replace(",", ".")
            return float(ah)/1000
        pattern = re.compile(r'([0-9.,]*\Sah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-2].replace(",", ".")
            return float(ah)
        pattern = re.compile(r'([0-9.,]* ah)', re.IGNORECASE)
        if (pattern.search(string) != None):
            ah = pattern.search(string).groups()[0][:-3].replace(",", ".")
            return float(ah)

class page_class:
    def __init__(self, url_request):
        htmlparser = etree.HTMLParser()
        page_ads = urlopen(url_request)
        self.tree_ads = etree.parse(page_ads, htmlparser)
    def get_infos_ad(self, i):
        ad_glob = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a")
        ad_url = "http:" + ad_glob[0].values()[0]
        ad_title = ad_glob[0].values()[1]

        ad_img = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/div[1]/span[1]/span[1]")
        url_img_ad = "http:" + ad_img[0].values()[2]

        ad_city_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/section/p[2]")
        ad_city = ad_city_elmt[0].getchildren()[0].values()

        ad_price_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[" + str(i) + "]/a/section/h3")
        ad_price = ad_price_elmt[0].values()[2]

        ad_time_elmt = self.tree_ads.xpath("/html/body/section[1]/main/section/section/section/section/ul/li[1]/a/section/aside/p")
        ad_time = ad_time_elmt[0].values()[2]
        
        return ad_class(ad_title, ad_price, ad_url, ad_time, ad_city, url_img_ad)
        
class detailed_page_class:
    def get_infos_ad(self):
        # we get the city
        xpath_city = "/html/body/section[1]/main/section/section/section/section/section/div[5]/h2/span[2]"
        city_elmt = self.tree_ad.xpath(xpath_city)
        city = city_elmt[0].text
        
        # we get the description
        # if there is only one image, there is only one section
        try:
            xpath_desc = "/html/body/section[1]/main/section/section/section/section/section[2]/div[6]/p[2]"
            desc_elmt = self.tree_ad.xpath(xpath_desc)
            text_iterator = desc_elmt[0].itertext()
            desc = ""
            for txt in text_iterator:
                desc = desc + " " + txt
        except:
            xpath_desc = "/html/body/section[1]/main/section/section/section/section/section/div[6]/p[2]"
            desc_elmt = self.tree_ad.xpath(xpath_desc)
            text_iterator = desc_elmt[0].itertext()
            desc = ""
            for txt in text_iterator:
                desc = desc + " " + txt
                
        # to get the images :
        # they are protected : a javascript code creates the link, so we have no direct access to the urls
        # we have to use selenium
        # !!!!! fazer

        return city,  desc, 1
    def __init__(self, url_ad):
        htmlparser = etree.HTMLParser()
        page_ad = urlopen(url_ad)
        self.tree_ad = etree.parse(page_ad, htmlparser)
        self.city, self.desc, self.images = self.get_infos_ad()
        
        
    

In [2]:

#page_ad = urlopen("https://www.leboncoin.fr/informatique/986006790.htm?ca=16_s")
#page_ad = urlopen("https://www.leboncoin.fr/informatique/1103074424.htm?ca=16_s")


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Firefox()
driver.get("https://www.leboncoin.fr/informatique/1103074424.htm?ca=16_s")


In [5]:
# only if there is more than 1 image
i = 2
xpath = "/html/body/section[1]/main/section/section/section/section/section[1]/div/ul/li[" + str(i) + "]/span/img"
elem = driver.find_element_by_xpath(xpath)
url_img = elem.get_attribute("src")
print(url_img)
driver.close()

https://img1.leboncoin.fr/ad-thumb/bcf9377259612bf7f7c11220d7c92248221315e5.jpg


In [154]:

# === loop on the pages
o = 1

url_request = "https://www.leboncoin.fr/annonces/offres/" + region + "/?o=" + str(o) + "&th=1&q=" + search + "&it=1" + location + "&parrot=0"
#page_ads = urlopen(url_request)
#tree_ads = etree.parse(page_ads, htmlparser)
page = page_class(url_request)


In [64]:
# == loop on the ads
ok = True
i = 0
while(ok):
    i = i + 1
    try:
        ad_infos = page.get_infos_ad(i)
        if (ad_infos.ah != None):
            print(ad_infos.ah / float(ad_infos.price), ad_infos.url)
    except:
        ok = False

['__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '_init', 'addnext', 'addprevious', 'append', 'attrib', 'base', 'clear', 'cssselect', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'getnext', 'getparent', 'getprevious', 'getroottree', 'index', 'insert', 'items', 'iter', 'iterancestors', 'iterchildren', 'iterdescendants', 'iterfind', 'itersiblings', 'itertext', 'keys', 'makeelement', 'nsmap', 'prefix', 'remove', 'replace', 'set', 'sourceline', 'tag', 'tail', 'text', 'values', 'xpath']
['__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir